Importing the libraries

In [0]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

Importing the dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

dataset = pd.read_table('/content/gdrive/My Drive/Dataset/car.data', sep = ',', names = ['buying', 'maint', 'doors', 'persons', 'lugboot', 'safety', 'class'])
print(dataset.info())

Histogram showing distribution of classes

In [0]:
print(dataset['class'].value_counts())
dataset['class'].value_counts().plot(kind='bar', title='Count (target)');

As you can see the dataset is highly imbalanced.
This will result in overfitting

Oversampling


In [0]:
class_count = dataset['class'].value_counts() #Class counts
max_class = max(class_count) #Getting the class with the max count i.e unacc

#Dividing dataset by class
dataset_class_0 = dataset[dataset['class']=='acc']
dataset_class_1 = dataset[dataset['class']=='good']
dataset_class_2 = dataset[dataset['class']=='unacc']
dataset_class_3 = dataset[dataset['class']=='vgood']

#Oversampling
dataset_class_0_over = dataset_class_0.sample(max_class, replace = True)
dataset_class_1_over = dataset_class_1.sample(max_class, replace = True)
dataset_class_3_over = dataset_class_3.sample(max_class, replace = True)

dataset_new = pd.concat([dataset_class_0_over,dataset_class_1_over,dataset_class_3_over,dataset_class_2], axis = 0)
dataset_new['class'].value_counts().plot(kind='bar', title='Count (target)');
print(dataset_new)

Preprocessing

In [0]:
X = dataset_new.drop(labels='class', axis=1)
Y = dataset_new['class']

#Encoding Categorical data
encoder_X = LabelEncoder()
for col in X.columns:
    X[col] = encoder_X.fit_transform(X[col])

encoder_y = LabelEncoder()
Y = encoder_y.fit_transform(Y)

#Getting Dummy Variables
X = pd.get_dummies(X, columns=X.columns, drop_first=True)

#Splitting the dataset into training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 42)

#Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#OneHotEncoding the target
one_hot_Y_train = to_categorical(Y_train) #For ANN

Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
#Fitting Logistic Regression to the training set
classifier = LogisticRegression(random_state=0, max_iter=1000)
classifier.fit(X_train,Y_train)

#Predicting the test set results
Y_pred = classifier.predict(X_test)

#Making the Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
acc_score = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test,Y_pred)
print("Accuracy: %0.2f\n" %(acc_score))
print('Classification Report:\n{}\n'.format(report))
print('Confusion Matrix:\n{}\n'.format(cm))

Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
#Fitting Naive Bayes to the training set
classifier = GaussianNB()
classifier.fit(X_train,Y_train)

#Predicting the test set results
Y_pred = classifier.predict(X_test)

#Making the Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
acc_score = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test,Y_pred)
print("Accuracy: %0.2f\n" %(acc_score))
print('Classification Report:\n{}\n'.format(report))
print('Confusion Matrix:\n{}\n'.format(cm))

Kernel SVM

In [0]:
from sklearn.svm import SVC
#Fitting Kernel SVM to the training set
classifier = SVC(kernel='rbf', random_state = 0, verbose=True)
classifier.fit(X_train,Y_train)

#Predicting the test set results
Y_pred = classifier.predict(X_test)

#Making the Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
acc_score = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test,Y_pred)
print("Accuracy: %0.2f\n" %(acc_score))
print('Classification Report:\n{}\n'.format(report))
print('Confusion Matrix:\n{}\n'.format(cm))

K-Nearest Neighbours

In [0]:
from sklearn.neighbors import KNeighborsClassifier
#Fitting KNN to the training set
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
classifier.fit(X_train,Y_train)

#Predicting the test set results
Y_pred = classifier.predict(X_test)

#Making the Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
acc_score = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test,Y_pred)
print("Accuracy: %0.2f\n" %(acc_score))
print('Classification Report:\n{}\n'.format(report))
print('Confusion Matrix:\n{}\n'.format(cm))

Decision Tree

In [0]:
from sklearn.tree import DecisionTreeClassifier
#Fitting Decision Tree to the training set
classifier = DecisionTreeClassifier(criterion='entropy',random_state=0)
classifier.fit(X_train,Y_train)

#Predicting the test set results
Y_pred = classifier.predict(X_test)

#Making the Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
acc_score = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test,Y_pred)
print("Accuracy: %0.2f\n" %(acc_score))
print('Classification Report:\n{}\n'.format(report))
print('Confusion Matrix:\n{}\n'.format(cm))

Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
#Fitting Random Forest to the training set
classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0, verbose=0, n_jobs=-1)
classifier.fit(X_train,Y_train)

#Predicting the test set results
Y_pred = classifier.predict(X_test)

#Making the Confusion Matrix
cm = confusion_matrix(Y_test, Y_pred)
acc_score = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test,Y_pred)
print("Accuracy: %0.2f\n" %(acc_score))
print('Classification Report:\n{}\n'.format(report))
print('Confusion Matrix:\n{}\n'.format(cm))

Creating an Artificial Neural Network

In [0]:
def create_model():
  #Initializing the ANN
  model = Sequential()

  #Adding layers to ANN
  model.add(Dense(8, kernel_initializer='uniform', activation='relu', input_shape = (15,))) #Input and First Hidden Layer
  model.add(Dense(8, kernel_initializer='uniform', activation='relu')) #Second Hidden Layer
  model.add(Dense(4, kernel_initializer='uniform', activation='softmax'))

  #Compiling the ANN
  model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy', 'mse'])
  return model

Fitting the model on the training set and predicting results on the test set

In [0]:
#Fitting the ANN to the training set
model = create_model()
model.fit(x = X_train, y = one_hot_Y_train, batch_size = 10,epochs = 100, verbose = 1)

#Predicting the test set results
y_pred = model.predict(X_test, verbose = 1)
y_pred = np.argmax(y_pred, axis=1)

#Confusion matrix
cm = confusion_matrix(Y_test, y_pred)
print(cm)

Evaluation using 10-Fold Cross Validation (k-Fold)

In [0]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
results = cross_val_score(model, X_train, Y_train, cv=kfold)
print('Average Accuracy: %0.2f' %(results.mean()))